# Diplodatos Kaggle Competition

We present this peace of code to create the baseline for the competition, and as an example of how to deal with these kind of problems. The main goals are that you:

1. Learn
1. Try different models and see which one fits the best the given data
1. Get a higher score than the given one in the current baseline example
1. Try to get the highest score in the class :)

In [1]:
# Import the required packages
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_columns', 20000)
pd.set_option('display.max_colwidth', -1)
import warnings
warnings.filterwarnings("ignore")

<ipython-input-1-a6820bab1e6a>:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


Read the *original* dataset...

In [2]:
def transform_data(train_data_fname, test_data_fname):
    df_train = pd.read_csv(train_data_fname)
    df_train['is_train_set'] = 1
    df_test = pd.read_csv(test_data_fname)
    df_test['is_train_set'] = 0
    
    # Comenzamos con los datos de TRAIN
    # -------------------------------------------------------------------------
    
    # Eliminamos los datos que tienen Nan en el Department Description
    clean_df = df_train.dropna(subset=['DepartmentDescription'])

    # Los datos que tienen Nan en Upc son todos de Pharmacy RX
    # Luego los completamos con esos valores, obtenidos por inspección
    clean_df.loc[clean_df.Upc.isna(), "FinelineNumber"] = 4822
    clean_df.loc[clean_df.Upc.isna(), "Upc"] = 30169183702
    
#    unique_tt = df_train.TripType.unique()
#    lista_df = []
#    for i in unique_tt:
#        iterator = clean_df[clean_df.TripType == i]
#        max = iterator.Upc.value_counts().idxmax()
#        iterator.Upc = max
#        lista_df.append(iterator)

    unique_vn = df_train.VisitNumber.unique()
    lista_df = []
    for i in unique_vn:
        iterator = clean_df[clean_df.VisitNumber == i]
        min = iterator.Upc.min()
        iterator.Upc = min
        lista_df.append(iterator)


    df_total = pd.concat(lista_df)
    df_total = df_total.drop(["FinelineNumber"], axis=1)
    y = df_total.groupby(["VisitNumber", "Weekday"], as_index=False).max().TripType

    df_total = df_total.drop("TripType", axis=1)
    
    # VISITNUMBER, DEPARTMENT, UPC, WEEKDAY
    
    #--------------------------------------------------------------------------
    
    df_test.loc[df_test.DepartmentDescription.isna(), "Upc"] = 0
    df_test.loc[df_test.Upc.isna(), "Upc"] = 30169183702
    unique_vn = df_test.VisitNumber.unique()
    lista_df = []
    for i in unique_vn:
        iterator = df_test[df_test.VisitNumber == i]
        min = iterator.Upc.min()
        iterator.Upc = min
        lista_df.append(iterator)

    test_total = pd.concat(lista_df)
    
    test_total = test_total.drop(["FinelineNumber"], axis=1)
    
    # VISITNUMBER, DEPARTMENT, UPC, WEEKDAY
    
    temp_concat = pd.concat([df_total, test_total])
    temp_concat = pd.get_dummies(temp_concat, columns=["DepartmentDescription"], dummy_na=True)
    temp_concat = temp_concat.groupby(["VisitNumber", "Weekday", "Upc"], as_index=False).sum()
    
    temp_concat = pd.get_dummies(temp_concat, columns=["Weekday"], dummy_na=True)
    
    df_train = temp_concat[temp_concat.is_train_set != 0]
    df_test = temp_concat[temp_concat.is_train_set == 0]

    X = df_train.drop(["is_train_set"], axis=1)
    yy = None
    XX = df_test.drop(["is_train_set"], axis=1)


    return X, y, XX, yy

Load the data...

In [3]:
X, y, XX, yy = transform_data("../data/train.csv", "../data/test.csv")

Create the model and evaluate it

In [4]:
# split training dataset into train and "validation" 
# (we won't be using validation set in this example, because of the cross-validation;
# but it could be useful for you depending on your approach)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
# results dataframe is used to store the computed results
results = pd.DataFrame(columns=('clf', 'best_acc'))

In [6]:
# we will use a DesicionTree to classify and GridSearch to determine the parameters
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

tree_param = {'criterion':('gini', 'entropy'), 'min_samples_leaf':(1, 2, 5),
              'min_samples_split':(2, 3, 5, 10, 50, 100)}
tree = DT(random_state=42)
tree_clf = GridSearchCV(tree, tree_param, cv=3, scoring='accuracy') #scoring='balanced_accuracy')
tree_clf.fit(X_train, y_train)
best_tree_clf = tree_clf.best_estimator_

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'criterion': ('gini', 'entropy'),
                         'min_samples_leaf': (1, 2, 5),
                         'min_samples_split': (2, 3, 5, 10, 50, 100)},
             scoring='accuracy')

In [7]:
print('Best Decision Tree accuracy: ', tree_clf.best_score_)
print(best_tree_clf)
results = results.append({'clf': best_tree_clf, 'best_acc': tree_clf.best_score_}, ignore_index=True)

print('The best classifier so far is: ')
print(results.loc[results['best_acc'].idxmax()]['clf'])

Best Decision Tree accuracy:  0.6420445607832895
DecisionTreeClassifier(min_samples_leaf=2, min_samples_split=100,
                       random_state=42)
The best classifier so far is: 
DecisionTreeClassifier(min_samples_leaf=2, min_samples_split=100,
                       random_state=42)


**And finally**, we predict the unknown label for the testing set

In [8]:
X.shape, XX.shape

((66917, 80), (28645, 80))

In [9]:
yy = results.clf.iloc[0].predict(XX)

The last thing we do is generating a file that should be *submitted* on kaggle

In [10]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, yy)), columns=["VisitNumber", "TripType"])

In [11]:
submission.to_csv("../data/submission.csv", header=True, index=False)

### Cosas para hacer:
##### Eliminar los Nan
##### Corroborar que los triptype para los registros de la misma compra dan igual
##### Relación entre finenumber y upc